In [1]:
import pandas as pd
import pickle
import datetime

import argparse
import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

# dya
spark = SparkSession \
    .builder \
    .config("spark.hadoop.hive.exec.dynamic.partition", "true") \
    .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

sc.setLogLevel("ERROR")
spark.sql('set spark.sql.caseSensitive=true')
spark.conf.set("spark.sql.hive.convertMetastoreParquet", "false")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [29]:
# data_date = datetime.datetime.today()- datetime.timedelta(1)
# year= data_date.year
# month = data_date.month
# day = data_date.day
data_date_str = (datetime.datetime.today()- datetime.timedelta(1)).strftime("%Y-%m-%d")

pay = pd.read_parquet(f"/data/gangyanyuan/data_rr/user_pay_movie1w/data_date={data_date_str}")
pay['bucket'] = pay.dnum.apply(lambda x:x[-1])
pay['user_aid_income'] = pay.user_aid_income.astype(float)
pay['sd_income'] = pay.sd_income.astype(float)

[Stage 88:====>                                                                                                                            (90 + 8) / 2548]

#6号桶付费情况

In [18]:
pay[pay.bucket.isin(['6'])]

,dnum,vid,title,bucket_cate,resourcetype,algo_type,paymode,clicktime,pay_time,user_clk_cnt,user_aid_income,income,sd_income,ecpm,eps_pv,eps_uv,clkc_pv,ctr,bucket
1,516672266,76n6vdky9nfocyg,阿凡达：水之道,autoIntV3,推荐算法,ManualHot,2.0,2023-08-09 04:55:39,2023-08-09 05:03:30,1,9.0,9.00,9.0,5.9682,1508,1214,14,0.0093,6
4,559847986,fhe2h7sop52qzza,疯狂动物城,autoIntV3,推荐算法,ManualHot,5.0,2023-08-09 09:45:09,2023-08-09 09:53:41,1,15.0,15.00,15.0,5.3841,2786,2093,77,0.0276,6
8,558903916,bv8m91i5jj1w1ww,冰雪奇缘,autoIntV3,推荐算法,ManualHot,5.0,2023-08-08 23:31:26,2023-08-08 23:36:00,1,19.0,19.00,19.0,3.9200,4847,3654,128,0.0264,6
18,437125926,fhe2h7sop52qzza,疯狂动物城,autoIntV3,推荐算法,-,5.0,2023-08-09 11:40:22,2023-08-09 11:47:15,1,68.0,68.00,68.0,49.1685,1383,922,32,0.0231,6


#对应一天的用户的点击记录(主要是为了看历史点击数据)

In [19]:
click_df = spark.sql(f"select * from dw_temp.xuyh_launcher_click WHERE  \
                      (data_date = '{data_date_str}') and mediatype=='电影'")
click_df[click_df.dnum=='516672266'].groupby(['data_date','vid','dnum','resourcelayout','algo_type']).agg(F.collect_set("title")).show()

+----------+---------------+---------+--------------+---------+------------------+
| data_date|            vid|     dnum|resourcelayout|algo_type|collect_set(title)|
+----------+---------------+---------+--------------+---------+------------------+
|2023-08-09|76n6vdky9nfocyg|516672266|   {30,0,35,7}|ManualHot|  [阿凡达：水之道]|
+----------+---------------+---------+--------------+---------+------------------+



In [23]:
click_df = spark.sql(f"select * from dw_temp.xuyh_launcher_click WHERE  \
                      (data_date = '{data_date_str}')")
click_df[click_df.dnum=='516672266'].groupby(['data_date','vid','dnum','resourcelayout','algo_type']).agg(F.collect_set("title")).show()

+----------+--------------------+---------+--------------+----------+--------------------+
| data_date|                 vid|     dnum|resourcelayout| algo_type|  collect_set(title)|
+----------+--------------------+---------+--------------+----------+--------------------+
|2023-08-08|CCTCLxMrbY1612921...|516672266|    {0,0,11,4}| jxTop-310|[汪汪队立大功 第2季]|
|2023-08-08|DGFAIDIqGH1600322...|516672266|     {3,0,5,3}|offline-v1|    [爆笑虫子无人岛]|
|2023-08-08|                   -|516672266|   {20,4,23,7}|         -|          [全部动画]|
+----------+--------------------+---------+--------------+----------+--------------------+



In [30]:
expose_df = spark.sql(f"select * from dw_temp.xuyh_launcher_expose WHERE  \
                      (data_date = '{data_date_str}') and mediatype=='电影'")
expose_df[expose_df.dnum=='516672266'].groupby(['data_date','vid','dnum','resourcelayout','algo_type']).agg(F.collect_set("title")).show()

+----------+------------------+---------+--------------+---------+--------------------------+
| data_date|               vid|     dnum|resourcelayout|algo_type|        collect_set(title)|
+----------+------------------+---------+--------------+---------+--------------------------+
|2023-08-09|   mzc00200kzadcff|516672266|   {24,0,29,7}|ManualHot|                [龙马精神]|
|2023-08-09|156264998568561502|516672266|   {18,0,35,7}|        -|        [狙击手：特别行动]|
|2023-08-09|   bv8m91i5jj1w1ww|516672266|   {12,0,17,7}|ManualHot|                [冰雪奇缘]|
|2023-08-09|   mzc00200yt0bu15|516672266|    {0,0,11,7}|        -|                [长空之王]|
|2023-08-09|   mzc00200hrjy2c7|516672266|   {12,0,23,7}|  AutoInt|[《八角笼中》小人物奋斗史]|
|2023-08-09|   fhe2h7sop52qzza|516672266|   {18,0,23,7}|ManualHot|              [疯狂动物城]|
|2023-08-09|   76n6vdky9nfocyg|516672266|   {30,0,35,7}|ManualHot|          [阿凡达：水之道]|
+----------+------------------+---------+--------------+---------+--------------------------+



#被点击电影的相关信息

In [67]:
click_df[click_df.title=='满江红'].show()

[Stage 170:================================================================================================================>                 (13 + 2) / 15]

+---------+---------------+--------------------+--------------+-----+-------+---------+----------------------+----------+--------------+------------+---------------+------+-------+---------+-----------+---------+--------+---------------+------------+-------+--------+-------------------------------+--------+------------+------------+--------+-----------+-------------------+--------------+--------+--------+----+--------------+----------+
|     dnum|     devicetype|          appversion|appversioncode|tabid|tabname|columnsid|           columnsname|resourceid|resourcelayout|resourcename|            vid| title|paymode|mediatype|bucket_cate|algo_type|block_id|columnsposition|recommend_id|test_id|group_id|                recommend_scene|position|resourcecate|resourcetype|business|subbusiness|          clicktime|      trace_id|tabindex|province|city|column_sorting| data_date|
+---------+---------------+--------------------+--------------+-----+-------+---------+----------------------+----------

#表中有数据，则使用下面这种方法展示曝光和点击数据

In [15]:
# expose_df = []
# click_df = []
# for i in range(1,6):
#     try:
#         data_date = (datetime.datetime.today()- datetime.timedelta(i)).strftime("%Y-%m-%d")
#         print(data_date)
#         click = pd.read_parquet(f"/data/gangyanyuan/data_rr/ln_user_click_movie10000/data_date={data_date}")
#         click['date'] = data_date
#         click_df.append(click)
#     except Exception as e:
#         pass

# click_df = pd.concat(click_df)


# for i in range(1,6):
#     try:
#         data_date = (datetime.datetime.today()- datetime.timedelta(i)).strftime("%Y-%m-%d")
#         print(data_date)
#         expose = pd.read_parquet(f"/data/gangyanyuan/data_rr/ln_user_expose_movie10000/data_date={data_date}")
#         expose['date'] = data_date
#         expose_df.append(expose)
     
#     except Exception as e:
#         pass
# expose_df = pd.concat(expose_df)

2023-08-06
2023-08-05
2023-08-04
2023-08-03
2023-08-02
2023-08-05
2023-08-04
2023-08-03
2023-08-02


In [12]:
expose_df[expose_df.dnum=='532875566'].groupby(['date','dnum','resourcelayout','algo_type']).agg({"title":"unique"})

NameError: name 'expose_df' is not defined

In [13]:
click_df[click_df.dnum=='532875566'].groupby(['date','dnum','resourcelayout','algo_type']).agg({"title":"unique"})

AnalysisException: "Undefined function: 'unique'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.;"

In [131]:
expose_df[expose_df.dnum=='587747636'].groupby(['date','dnum','resourcelayout','algo_type']).agg({"title":"unique"})

title
date       dnum      resourcelayout algo_type            
2023-08-02 587747636 {0,8,5,15}     ClickHot        [狙击手]
                     {12,0,17,7}    ManualHot      [检察风云]
                     {12,8,17,15}   ClickHot       [扫黑行动]
                     {18,0,23,7}    HistSim       [东北合伙人]
                     {18,8,23,15}   HistSim       [李茂扮太子]
                     {24,0,29,7}    ManualHot   [阿凡达：水之道]
                     {24,8,29,15}   ClickHot        [东北虎]
                     {30,0,35,7}    HistSim      [东北告别天团]
                     {6,8,11,15}    HistSim    [天才不能承受之重]

In [142]:
click_df[click_df.dnum=='587747636'].groupby(['date','dnum','resourcelayout','algo_type']).agg({"title":"unique"})

,,,,title
date,dnum,resourcelayout,algo_type,
2023-08-02,587747636,"{24,0,29,7}",ManualHot,[阿凡达：水之道]


#用户连续两天有点击，第三天付费

In [37]:
user_play = pd.read_parquet("/data/gangyanyuan/data/user_latest_play_movie10000_plus_30days/year=2023/month=8/day=9/bucket=6/second_bucket=6")

In [38]:
user_play[user_play.dnum=='516672266']

,dnum,all_play_item,all_types_list,all_play_action_date,all_play_diff_date,all_played_duration,all_types_list_1,all_types_list_2
19814,516672266,"[76n6vdky9nfocyg, PAD, PAD, PAD, PAD, PAD, PAD...","[[冒险, 科幻], [PAD, PAD], [PAD, PAD], [PAD, PAD],...","[2023-08-09, PAD, PAD, PAD, PAD, PAD, PAD, PAD...","[0, PAD, PAD, PAD, PAD, PAD, PAD, PAD, PAD, PA...","[360, PAD, PAD, PAD, PAD, PAD, PAD, PAD, PAD, ...","[冒险, PAD, PAD, PAD, PAD, PAD, PAD, PAD, PAD, P...","[科幻, PAD, PAD, PAD, PAD, PAD, PAD, PAD, PAD, P..."


In [41]:
media = pd.read_parquet("/data/gangyanyuan/data_rr/movie_10000_plus_media/20230802.parquet")
media_movie = pd.read_parquet("/data/gangyanyuan/data_rr/doubaninfo_detail_merged_rr_v2")

In [45]:
media_movie.columns

Index(['aid', 'title', 'channel_name', 'vendor_list', 'quality', 'is_pay',
       'nation', 'year', 'types', 'summarys', 'directors', 'score', 'language',
       'doubanrecommendations', 'doubanduration', 'operate_status',
       'content_type', 'tags', 'actors', 'age_min', 'age_max'],
      dtype='object')

In [14]:
media[media.vid.isin(['mzc00200ln1dq42','bv8m91i5jj1w1ww'])]

,id,pool_id,name,description,title,background_1080,url_image_size,org_url_image_size,action_type,action_url,...,user_name,create_time,video_type,video_type_name,states,uuid,multi_size_poster,aid_ispay,is_new,date
343,721073,40,冰雪奇缘,,"["""",""""]",http://puui.qpic.cn/vcover_vt_pic/0/bv8m91i5jj...,189,189,2,null,...,高扬,yang28.gao,2023-07-13 14:21:44.0,01001,电影,null,1,5,False,2023-08-02


 #被点击电影的信息
 is_pay = 5, 会员免费
 is_pay =2 单点付费
 is_pay =1 免费

In [31]:
media_movie[media_movie.aid=='76n6vdky9nfocyg'][['aid','title','is_pay','types','tags','actors','directors','year','nation','language']]

,aid,title,is_pay,types,tags,actors,directors,year,nation,language
54132,76n6vdky9nfocyg,阿凡达：水之道,2,"[冒险, 科幻, 动作]","[院线, 阿凡达2]","[萨姆·沃辛顿, 佐伊·索尔达娜, 西格妮·韦弗, 史蒂芬·朗, 克利夫·柯蒂斯, 乔·大卫...",[詹姆斯·卡梅隆],2022,[美国],[英语]


In [32]:
media_movie[media_movie.aid=='bv8m91i5jj1w1ww'][['aid','title','is_pay','types','tags','actors','directors','year','nation','language']]

,aid,title,is_pay,types,tags,actors,directors,year,nation,language
13590,bv8m91i5jj1w1ww,冰雪奇缘,5,"[奇幻, 歌舞, 喜剧, 音乐, 动画]","[情感, 院线, 迪士尼, 迪斯尼, 白雪皇族, 魔雪奇缘, 冰雪皇族]",None,"[克里斯·巴克, 珍妮弗·李]",2013,[美国],[英语]


In [33]:
media_movie[media_movie.aid=='fhe2h7sop52qzza'][['aid','title','is_pay','types','tags','actors','directors','year','nation','language']]

,aid,title,is_pay,types,tags,actors,directors,year,nation,language
46907,fhe2h7sop52qzza,疯狂动物城,5,"[搞笑, 喜剧, 冒险, 励志, 动画]","[院线, 动物, 迪士尼]",None,"[拜伦·霍华德, 里奇·摩尔, 杰拉德·布什]",2016,[美国],[英语]


In [34]:
media_movie[media_movie.aid=='mzc00200kzadcff'][['aid','title','is_pay','types','tags','actors','directors','year','nation','language']]

,aid,title,is_pay,types,tags,actors,directors,year,nation,language
3182,mzc00200kzadcff,龙马精神,5,"[剧情, 喜剧, 动作]",[院线],"[成龙, 刘浩存, 郭麒麟, 吴京, 余皑磊, 容祖儿, 于荣光, 小沈阳, 潘斌龙, 贾冰...",[杨子],2023,[大陆],[普通话]


#找一下用户前N=2天.联系有点击行为的媒资，第三天付费

In [87]:
user_click_ndays = click_df[click_df.date<='2023-08-07'].groupby(['dnum','title']).agg({"date":"nunique"}).reset_index()
user_click_ndays = user_click_ndays[user_click_ndays.date>=2]

AttributeError: 'DataFrame' object has no attribute 'date'

In [191]:
user_found = pd.merge(pay, user_click_ndays, on=['dnum','title'])
user_found

,dnum,vid,title,bucket_cate,resourcetype,algo_type,paymode,clicktime,pay_time,user_clk_cnt,user_aid_income,income,sd_income,ecpm,eps_pv,eps_uv,clkc_pv,ctr,bucket,date
0,573948827,bv8m91i5jj1w1ww,冰雪奇缘,hot-basic,人工运营,-,5.0,2023-08-02 23:39:45,2023-08-02 23:50:44,1,9.9,9.90,9.9,3.8417,2577,1681,66,0.0256,7,2


23/08/04 15:27:12 ERROR [dispatcher-event-loop-4] YarnScheduler: Lost executor 1 on emr-worker-71415.cluster-237382: Container from a bad node: container_e30_1679576612862_1704053_01_000004 on host: emr-worker-71415.cluster-237382. Exit status: 143. Diagnostics: [2023-08-04 15:27:12.636]Container killed on request. Exit code is 143
[2023-08-04 15:27:12.636]Container exited with a non-zero exit code 143. 
[2023-08-04 15:27:12.638]Killed by external signal
.


In [18]:
user_found.groupby(['bucket_cate','resourcetype']).agg({"dnum":"nunique","user_aid_income":"sum"})

dnum  user_aid_income
bucket_cate resourcetype                       
autoIntV3   推荐算法             2             33.4
hot-basic   人工运营            13            356.0
            推荐算法             9            109.5
人工          人工运营             6             53.1

In [19]:
#有付费的用户，第三天的购买都是前两天有连续点击的媒资
user_found.user_aid_income.sum(),pay.user_aid_income.sum()

(552.0, 552.0)

In [20]:
#也有些用户连续两天点击，但是未购买
user_not_found = pd.merge(pay, user_click_ndays, on=['dnum','title'],how='right')
user_not_found[user_not_found.user_aid_income.isna()]

,dnum,vid,title,bucket_cate,resourcetype,algo_type,paymode,clicktime,pay_time,user_clk_cnt,user_aid_income,income,sd_income,ecpm,eps_pv,eps_uv,clkc_pv,ctr,bucket,date
0,106445692,NaN,冰雪奇缘,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,106445692,NaN,疯狂动物城,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,115504745,NaN,狩猎,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,150858127,NaN,检察风云,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,150859656,NaN,扫黑行动,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11544,643003110,NaN,满江红,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11545,643015279,NaN,幸存者1937,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11546,643015287,NaN,幸存者1937,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11547,643015308,NaN,幸存者1937,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
